In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
train=pd.read_csv("train.csv", parse_dates=['datetime'])
test=pd.read_csv("test.csv", parse_dates=['datetime'])

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
datetime      6493 non-null datetime64[ns]
season        6493 non-null int64
holiday       6493 non-null int64
workingday    6493 non-null int64
weather       6493 non-null int64
temp          6493 non-null float64
atemp         6493 non-null float64
humidity      6493 non-null int64
windspeed     6493 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 456.7 KB


In [4]:
train['year']=train['datetime'].dt.year
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.day
train['hour']=train['datetime'].dt.hour
train['dayofweek']=train['datetime'].dt.dayofweek
test['year']=test['datetime'].dt.year
test['month']=test['datetime'].dt.month
test['day']=test['datetime'].dt.day
test['hour']=test['datetime'].dt.hour
test['dayofweek']=test['datetime'].dt.dayofweek

In [5]:
#train['windspeed']=np.where(train['windspeed']==0, np.nan, train['windspeed'])
#train['windspeed']=np.where(train['windspeed'].isnull(), train[train['windspeed'].notnull()]['windspeed'].mean(), train['windspeed'])

In [6]:
#test['windspeed']=np.where(test['windspeed']==0, np.nan, test['windspeed'])
#test['windspeed'].fillna(test['windspeed'].mean(), inplace=True)

In [7]:
train['windspeed']
test['windspeed']

0       26.0027
1        0.0000
2        0.0000
3       11.0014
4       11.0014
         ...   
6488    11.0014
6489    11.0014
6490    11.0014
6491     8.9981
6492     8.9981
Name: windspeed, Length: 6493, dtype: float64

In [8]:
# 머신러닝(랜덤포레스트)을 이용한 풍속 0값을 대체
# 풍속 0, not0으로 분류
trainWind0=train.loc[train.windspeed==0]
trainWindNot0=train.loc[train.windspeed!=0]

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
def predictWindSpeed(data):
    # data의 windspeed열 값 0을 랜덤포레스트 기반 예측값으로 대체
    dataWind0=data.loc[data.windspeed==0]
    dataWindNot0=data.loc[data.windspeed!=0]
    # 입력데이터() -> 랜덤포레스트 모델 -> 출력데이터(windspeed)
    # 풍속을 예측하는데 사용될 변수(입력)를 선택
    wCol=['season', 'weather', 'temp', 'atemp', 'humidity', 'month']
    
    # 회귀모델
    # 풍속예측함수 = w1*season+w2*weathe+...+w7*month+b
    
    # 출력 데이터 타입이 str이어야 했음
    dataWindNot0['windspeed']=dataWindNot0['windspeed'].astype('str')
    # 랜덤포레스트 분류기
    rfModelWind=RandomForestClassifier()
    
    # wind not 0 데이터로 학습(fit)을 시켜서 모델을 만든 후, wind 0 데이터를 모델에 입력하면 예상되는 풍속이 출력된다.
    # 모델링(학습데이터)
    rfModelWind.fit(dataWindNot0[wCol], dataWindNot0['windspeed'])
    
    # 모델(rfModelWind)을 이용하여 풍속이 0인 데이터에 대한 풍속을 예측(predict)하자
    wind0values=rfModelWind.predict(dataWind0[wCol])
    predictWind0=dataWind0
    predictWindNot0=dataWindNot0
    
    # 풍속 0을 예측된 값으로 변경
    predictWind0['windspeed']=wind0values
    
    # 풍속이 0이 아닌 데이터프레임에 예측된 값이 저장된 데이터프레임을 합침
    data=predictWindNot0.append(predictWind0)
    
    data['windspeed']=data['windspeed'].astype(float)
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    
    return data

In [11]:
train=predictWindSpeed(train)
#test=predictWindSpeed(test)

C:\Users\msi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\msi\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
train['windspeed'].describe()

count    10886.000000
mean        14.027992
std          7.044150
min          6.003200
25%          8.998100
50%         12.998000
75%         19.001200
max         56.996900
Name: windspeed, dtype: float64

In [13]:
# 연속형 변수 -> 범주형 변수
category_fn=['season','holiday','workingday','weather','year','month','hour','dayofweek']

In [14]:
for v in category_fn:
    train[v]=train[v].astype('category')
    test[v]=test[v].astype('category')

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 17 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null category
holiday       10886 non-null category
workingday    10886 non-null category
weather       10886 non-null category
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
year          10886 non-null category
month         10886 non-null category
day           10886 non-null int64
hour          10886 non-null category
dayofweek     10886 non-null category
dtypes: category(8), datetime64[ns](1), float64(3), int64(5)
memory usage: 852.8 KB


In [16]:
from sklearn.metrics import make_scorer

In [17]:
def rmsle(pv,av):
    pv=np.array(pv)
    av=np.array(av)
    log_predict=np.log(pv+1)
    log_actual=np.log(av+1)
    res=log_predict-log_actual
    res=np.square(res)
    mean_res=res.mean()
    score=np.sqrt(mean_res)
    return score

In [18]:
rmsle_scorer=make_scorer(rmsle)

In [19]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [20]:
kfold=KFold(n_splits=10, shuffle=True, random_state=43)

In [21]:
from sklearn.ensemble import RandomForestRegressor

In [22]:
model=RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=43)

In [23]:
fn=['season','holiday','workingday','weather','month','hour','dayofweek', 'temp', 'atemp', 'humidity', 'windspeed']

In [24]:
xTrain=train[fn]

In [25]:
xTest=test[fn]

In [26]:
yTrain=train['count']

In [27]:
model.fit(xTrain, yTrain)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=43, verbose=0, warm_start=False)

In [28]:
pred=model.predict(xTest)
pred

array([ 16.18,   6.02,   5.37, ..., 124.48,  93.63,  56.76])

In [29]:
score=cross_val_score(model, xTrain, yTrain, cv=kfold, scoring=rmsle_scorer)
score.mean()

0.4019620166278717

In [30]:
score

array([0.41959196, 0.39312368, 0.40076395, 0.41394112, 0.41283018,
       0.43392177, 0.3852183 , 0.37087497, 0.41163439, 0.37771985])

In [31]:
bikesubmit=pd.read_csv("sampleSubmission.csv")
bikesubmit

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0
...,...,...
6488,2012-12-31 19:00:00,0
6489,2012-12-31 20:00:00,0
6490,2012-12-31 21:00:00,0
6491,2012-12-31 22:00:00,0


In [32]:
bikesubmit['count']=pred
bikesubmit

,datetime,count
0,2011-01-20 00:00:00,16.18
1,2011-01-20 01:00:00,6.02
2,2011-01-20 02:00:00,5.37
3,2011-01-20 03:00:00,3.75
4,2011-01-20 04:00:00,3.23
...,...,...
6488,2012-12-31 19:00:00,263.14
6489,2012-12-31 20:00:00,158.81
6490,2012-12-31 21:00:00,124.48
6491,2012-12-31 22:00:00,93.63


In [33]:
bikesubmit.to_csv("bikesubmit.csv", index=False)